In [ ]:
def load_data():
    """
    Load training data and split it into training and validation set
    """
    data_train = pd.read_csv('driving_log.csv', names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])
    X = data_train[['center', 'left', 'right']].values
    y = data_train['steering'].values

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=32)

    return X_train, X_valid, y_train, y_valid

In [ ]:
def Preprocess(image):
    
    image = mpimg.imread(image_file) #Load RGB images from a file
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV) # Convert the image from RGB to YUV (This is what the NVIDIA model does)
    image = image/255 # normalize pixels value between 0 & 1
    return image

In [ ]:
def batch_generator(X_train, y_train, batch_size):
    """
    Generate training image and associated steering angles
    """
    images = list()
    steers = list()

    while 1:
        batch_count = 0
        for index in range(X_train.shape[0]):
        
            center, left, right = X_train[index]
            steering_angle = y_train[index]
            
            # image preprocess
            center_image = Preprocess(center)
            left_image = Preprocess(left)
            right_image = Preprocess(right)
                
            #add the image and steering angle to the batch
            images.append(center_image)
            images.append(left_image)
            images.append(right_image)
            
            steers.append(steering_angle)
            steers.append(steering_angle)
            steers.append(steering_angle)
            
            batch_count = batch_count + 1
            
            if batch_count == batch_size:
                batch_count = 0
                yield (np.array(images),np.array(steers))
                images = list(), steers = list()
            

In [ ]:
def batch_generator(X_train, y_train, batch_size):
    """
    Generate training image give image paths and associated steering angles
    """
    images = np.empty([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS])
    steers = np.empty(batch_size)
    #print(X_train.shape)
    while 1:
        n=0
        for index in np.random.permutation(X_train.shape[0]):
        
            center, left, right = X_train[index]
            steering_angle = y_train[index]
            
            # Preprocess
            image = Preprocess(center)
                
            # add the image and steering angle to the batch
            images[n] = preprocess(image)
            steers[n] = steering_angle
            n+=1
            if n == batch_size:
                break
        #print(images[0].shape)
        yield (images, steers)
            

In [ ]:
def train_model(model, data):
    """
    Train the model
    """
    #Saves the model after every epoch.
    #quantity to monitor, verbosity i.e logging mode (0 or 1), 
    #if save_best_only is true the latest best model according to the quantity monitored will not be overwritten.
    #mode: one of {auto, min, max}. If save_best_only=True, the decision to overwrite the current save file is
    
    checkpoint = ModelCheckpoint('model-{epoch:03d}.h5',
                                 monitor='loss',
                                 verbose=0,
                                 save_best_only=True,
                                 mode='auto')

    model.compile(loss='mean_squared_error', optimizer='Adam',metrics=['accuracy'])
    
    X_train, X_valid, y_train, y_valid = data
    
    batch_size = 10
    steps = X_train.shape[0]//batch_size
    
    #Fits the model on data generated batch-by-batch by a Python generator.
    generator = batch_generator(X_train, y_train, batch_size=10)
    
    model.fit(
                generator,
                epochs = 30,
                steps_per_epoch=steps,
                #validation_data=batch_generator(X_valid, y_valid, batch_size=10),
                #callbacks=[checkpoint],
             )
    return model